In this tutorial, we show how to prepare a dataset of human SAVs, with relative features and true labels, to be used for training a Rhapsody classifier.

## List of human missense variants 

We start by importing lists of human SAVs, with relative pathogenicity assessments, compiled from the following publicly available datasets:
* **HumVar, ExoVar, predictSNP, VariBench, SwissVar**, 5 datasets of labelled human missense variants already used in our  [previous publication](https://www.pnas.org/content/115/16/4164) 
* **Humsavar**, a database of "human polymorphisms and disease mutations" available on [Uniprot](https://www.uniprot.org/docs/humsavar)
* **ClinVar** a "public archive of reports of the relationships among human variations and phenotypes, with supporting evidence" [(FTP site)](ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/). It contains a *review score*, based on a ranking out of 4 stars, that allows us to test performances on different levels of "confidence in the accuracy of variation calls and assertions of clinical significance".

When combining these datasets together, SAVs with discordant interpretation are assigned with a `true_label = -1`.

The resulting dataset is called **Integrated Dataset**.

In [1]:
import os,sys
import numpy as np

In [2]:
import tarfile

# extract data
if not os.path.isdir('local'):
    os.mkdir('local')
if not os.path.isdir('local/data'):
    tar = tarfile.open('data.tar.gz', "r:gz")
    tar.extractall(path='local')
    tar.close()

In [3]:
# import numpy structured array containing list of SAVs
ID = np.load('local/data/Integrated_Dataset-SAVs.npy')

In [4]:
print('Integrated Dataset size:', len(ID))

Integrated Dataset size: 94505


In [5]:
print(ID.dtype.names)
print(ID[0])

('SAV_coords', 'true_label', 'datasets', 'ClinVar_review_star')
('A0AV02 181 R C', 0, 'humsavar[0],swissvar[0]', -1)


In [6]:
# true labels: 0 (neutral), 1 (deleterious), -1 (unknown or discordant interpretations)
print( set(ID['true_label']) )

{0, 1, -1}


SAVs imported from ClinVar are assigned with a review star (see this [link](https://www.ncbi.nlm.nih.gov/clinvar/docs/review_status) for meaning). If a SAV was not found in ClinVar and does not have a review star, we will set its value to `-1`.

In [7]:
print( set(ID['ClinVar_review_star']) )

{0, 1, 2, 3, 4, -1}


## Dataset composition

In [8]:
from collections import Counter
import re

list_of_datasets = ['clinvar', 'exovar', 'humsavar', 'humvar', 'predictSNP', 'swissvar', 'varibench']
print(' '*11 + ' '.join([f'{d:15}' for d in list_of_datasets]))

for i, d_i in enumerate(list_of_datasets):
    print(f'{d_i:11}', end='')
    for j, d_j in enumerate(list_of_datasets):
        if j < i:
            shared_SAVs = [s for s in ID['datasets'] if d_i in s and d_j in s]
            d_i_tlabels = [re.findall(d_i + '\[.\]', s)[0].split('[')[1][:-1] for s in shared_SAVs]
            d_j_tlabels = [re.findall(d_j + '\[.\]', s)[0].split('[')[1][:-1] for s in shared_SAVs]
            n_discordant = sum(np.array(d_i_tlabels) != np.array(d_j_tlabels))
            if n_discordant:
                summary = f'{len(shared_SAVs)}({n_discordant})'
            else:
                summary = f'{len(shared_SAVs)}'
            print(f'{summary:<15}', end=' ')
        elif j == i:
            SAVs = [s for s in ID['datasets'] if d_i in s]
            tl_count = Counter([re.findall(d_j + '\[.\]', s)[0].split('[')[1][:-1] for s in SAVs])
            n_ambiguous  = tl_count['?']
            dataset_bias = 100*tl_count['1']/(tl_count['0'] + tl_count['1'])
            if n_ambiguous:
                summary = f'{len(SAVs)}({n_ambiguous})/{dataset_bias:3.1f}%'
            else:
                summary = f'{len(SAVs)}/{dataset_bias:3.1f}%'
            print(f'{summary:<15}', end=' ')    
        else:
            print()
            break

           clinvar         exovar          humsavar        humvar          predictSNP      swissvar        varibench      
clinvar    20814(89)/61.4% 
exovar     2984(475)       8809/58.5%      
humsavar   19043(2729)     5098(341)       68386/42.4%     
humvar     12429(2222)     5438(21)        37003(1005)     40177/52.3%     
predictSNP 299(21)         8               3222(48)        25(1)           10459(1)/62.6%  
swissvar   1119(127)       10              7058(60)        56(1)           21(1)           8862(1)/31.7%   
varibench  665(291)        1               697(66)         14              2               4               10237/42.1%     

In parentheses: SAVs with discordant clinical interpretations between the two datasets.
On the diagonal, the *dataset bias* (percentage of positive cases) is also reported.

## Computing Rhapsody features
Precomputed features can be found in `local/data/` (computing them from scratch for the complete dataset takes a couple of days).
In the following, we show how to compute Rhapsody features for a small set of SAVs.

In [9]:
# If needed, insert here local path to Rhapsody folder with the command:
# sys.path.insert(0, '/LOCAL_PATH/rhapsody/')
import rhapsody as rhaps

In [10]:
test_SAVs = ['O00294 496 A T', 'O00238 31 R H']

In [11]:
# set folder where pickles used to speed up calculations will be stored
if not os.path.isdir('local/pickles'):
    os.mkdir('local/pickles')
rhaps.pathRhapsodyFolder('local/pickles/')

@> Local Rhapsody folder is set: '/home/lponzoni/Scratch/028-RHAPSODY-git/rhapsody-tutorials/paper_Supplementary_Info/00-Training_Dataset/local/pickles'


Precomputed "mutation effects" from EVmutation can be downloaded at this [link](https://marks.hms.harvard.edu/evmutation/human_proteins.html) (~460 MB).

In [12]:
# Insert here path to folder with EVmutation precomputed mutation effects
rhaps.pathEVmutationFolder('/home/lponzoni/Data/025-EVmutation/mutation_effects')

@> Local EVmutation folder is set: '/home/lponzoni/Data/025-EVmutation/mutation_effects'


In [13]:
if not os.path.isdir('local/results'):
    os.mkdir('local/results')
os.chdir('local/results')

In [14]:
# initialize a rhapsody object
rh = rhaps.Rhapsody()

In [15]:
# import SAVs by querying PolyPhen-2 (or by importing precomputed PolyPhen-2 output file, if found)
if os.path.isfile('pph2-full.txt'):
    rh.importPolyPhen2output('pph2-full.txt')
else:
    rh.queryPolyPhen2(test_SAVs)

@> PolyPhen-2's output parsed.


In [16]:
# we would like to compute all features
rh.setFeatSet('all')

In [17]:
# true labels must be imported prior to exporting training data
true_labels = {
    'O00294 496 A T': 1,
    'O00238 31 R H': 0
}
rh.setTrueLabels(true_labels)

In [18]:
training_dataset = rh.exportTrainingData()
training_dataset

@> Sequence-conservation features have been retrieved from PolyPhen-2's output.
@> Mapping SAVs to PDB structures...
@> [1/2] Mapping SAV 'O00238 31 R H' to PDB...
@> Pickle 'UniprotMap-O00238.pkl' recovered.
@> [2/2] Mapping SAV 'O00294 496 A T' to PDB...
@> Pickle 'UniprotMap-O00238.pkl' saved.
@> Pickle 'UniprotMap-O00294.pkl' recovered.
@> Pickle 'UniprotMap-O00294.pkl' saved.
@> SAVs have been mapped to PDB in 0.1s.
@> Computing structural and dynamical features from PDB structures...
@> [2/2] Analizing mutation site 2FIM:A 443...
@> Pickle 'PDBfeatures-2FIM.pkl' recovered.
@> Pickle 'PDBfeatures-2FIM.pkl' saved.
@> PDB features have been computed in 0.0s.
@> Computing sequence properties from Pfam domains...
@> [1/2] Mapping SAV 'O00238 31 R H' to Pfam...
@> Pickle 'UniprotMap-O00238.pkl' recovered.
@> [2/2] Mapping SAV 'O00294 496 A T' to Pfam...
@> Pickle 'UniprotMap-O00238.pkl' saved.
@> Pickle 'UniprotMap-O00294.pkl' recovered.
@> Pickle 'UniprotMap-O00294.pkl' saved.
@> SAVs

array([('O00294 496 A T', '2FIM A 443 A', 224, 1, 0.41869268, 0.2725254, 0.30990002, 0.00208266, 0.00289542, 0.0444308, 0.00296154, 0.00206255, 0.04027145, 0., 0.754,  1., -3.1479, -1.0065, 0.0662341, 0.3341, 0.09581726, 0.08719353, 0.08953744, 0.00398543, 0.00332497, 0.01278353, 0.00573205, 0.00480731, 0.01438932, 79., 78., 2.1440704, 0.813278  , 12.638108, 14.42894, 14.559032, -2.376),
       ('O00238 31 R H', 'Unable to map SAV to PDB',   0, 0,        nan,       nan,        nan,        nan,        nan,       nan,        nan,        nan,        nan, 0., 1.634, nan, -2.4718, -2.461 , 0.0102818, 0.3508,        nan,        nan,        nan,        nan,        nan,        nan,        nan,        nan,        nan, nan, nan, 1.8702421, 0.23376623,       nan,      nan,       nan, -2.769)],
      dtype=[('SAV_coords', '<U50'), ('Uniprot2PDB', '<U100'), ('PDB_length', '<i2'), ('true_label', '<i2'), ('ANM_MSF-chain', '<f4'), ('ANM_MSF-reduced', '<f4'), ('ANM_MSF-sliced', '<f4'), ('ANM_effectiven

In [19]:
np.save('precomputed_features', training_dataset)